In [1]:
from sx_multi import DataSource
from sx_multi import Analysis
from sx_multi import Accumulator
from sx_multi import LocalExecutor

from sx_multi import FuncXExecutor

import matplotlib.pyplot as plt



from servicex import ServiceXDataset
from sx_multi import FuncAdlDataset
from coffea import hist, processor


In [2]:
dids = ['mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00']
datasets = [
    ServiceXDataset(did, backend_type='xaod')
    for did in dids
]

In [3]:
leptons_per_event_query = FuncAdlDataset() \
        .Select(lambda e: e.Electrons("Electrons")) \
        .Select(lambda eles: eles.Where(lambda e: e.pt()/1000.0 > 30.0)) \
        .Select(lambda eles: eles.Where(lambda e: abs(e.eta()) < 2.5)) \
        .Where(lambda eles: len(eles) == 2) \
        .Select(lambda ls: (ls.Select(lambda e: e.pt()/1000.0), ls.Select(lambda e: e.eta()), ls.Select(lambda e: e.phi()), ls.Select(lambda e: e.m()/1000.0), ls.Select(lambda e: e.charge()))) \
        .AsROOTTTree('data.root', 'mytree', ('electrons_pt', 'electrons_eta', 'electrons_phi', 'electrons_mass', 'electrons_charge'))

In [4]:
datasource = DataSource(query=leptons_per_event_query, metadata={}, datasets=datasets)

In [5]:
class Z_EEAnalysis(Analysis):
    def __init__(self):
        self.accumulator = Accumulator({
            "mass": hist.Hist(
                "Events",
                hist.Cat("dataset", "Dataset"),
                hist.Bin("mass", "$Z_{ee}$ [GeV]", 60, 60, 120),
            ),
        })

    @staticmethod
    def process(output, events):
        from sx_multi import Accumulator
        import awkward1 as ak

        dataset = events.metadata['dataset']
        electrons = events.electrons

        # Form the invar mass, plot.
        cut = (ak.num(electrons) == 2)
        diele = electrons[cut][:, 0] + electrons[cut][:, 1]

        output["sumw"][dataset] += len(events)
        output["mass"].fill(
            dataset=dataset,
            mass=diele.mass,
        )

        return output



In [6]:
analysis = Z_EEAnalysis()
executor = FuncXExecutor(endpoint_id='d0a328a1-320d-4221-9d53-142189f77e71')
# executor = LocalExecutor()

Batch result:  {}


In [7]:
%matplotlib widget
async def plot_stream(accumulator_stream):
  async for coffea_info in accumulator_stream:
    # Need to ask coffea folks how to anomate this!
    hist.plot1d(coffea_info['mass'])
    plt.show()
  return coffea_info

await plot_stream(executor.execute(analysis, datasource))

Batch result:  {'ba8e9ae6-4878-4312-a8d9-10ae692c701e': {'pending': True, 'status': 'waiting-for-nodes'}, '6240cddd-d0f0-4813-aa0c-680214bd07d3': {'pending': True, 'status': 'waiting-for-nodes'}, 'f83a8dd6-7d5e-45df-979c-82eba6621dba': {'pending': True, 'status': 'waiting-for-nodes'}, '955e2a3f-d53f-4b70-8121-25eba0640ca7': {'pending': True, 'status': 'waiting-for-nodes'}, '1764340b-2643-4797-947c-479c12d98d49': {'pending': True, 'status': 'waiting-for-nodes'}, '401781c1-3dac-4ea6-ab8c-ebfc47bc21b2': {'pending': True, 'status': 'waiting-for-nodes'}, '96180a37-a9ea-4525-aca5-c0424fcb2e90': {'pending': True, 'status': 'waiting-for-nodes'}, '44742f5f-3342-4a0e-8992-1a96538dc241': {'pending': True, 'status': 'waiting-for-nodes'}, 'ecc000b4-3791-43fd-bb69-b06a2630e162': {'pending': True, 'status': 'waiting-for-nodes'}, '9cdc174f-e6a4-495b-b837-889e8f0dff3c': {'pending': True, 'status': 'waiting-for-nodes'}, '882fe73f-c58b-42f7-b529-3fba255813a7': {'pending': True, 'status': 'waiting-for-nod

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'mass': <Hist (dataset,mass) instance at 0x132ea3a90>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 50000.0})}
{'mass': <Hist (dataset,mass) instance at 0x132f00310>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 24000.0})}
{'mass': <Hist (dataset,mass) instance at 0x132edf190>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 50000.0})}
{'mass': <Hist (dataset,mass) instance at 0x132f00110>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 45000.0})}
Batch result:  {'6240cddd-d0f0-4813-aa0c-680214bd07d3': {'pending': False, 'status': 'success', 'result': {'mass': <Hist (dataset,mass) instance at 0x132f81550>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}, 'completion_t': '1606340248.4396546'}, 'f83a8dd6-7d5e-45df-979c-82eba6621dba': {'pending': False, 'status': 'success', 'result': {'mass': <Hist (dataset,mass) instance at 0x132f06e90>, 'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0})}, 'comp

{'mass': <Hist (dataset,mass) instance at 0x132cbda90>,
 'sumw': defaultdict_accumulator(float, {'mc15x': 1993800.0})}